In [24]:
import cv2
import imutils
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'c:\users\manu-\appdata\local\pip\cache\wheels\ed\8a\c6\40c7ec06c2dd3df636832537238128a1471bf9c1b6a3a9bf40\tesseract.exe'


In [25]:
for i in range(1, 20):

    img    = cv2.imread(f"archive/images/Cars{i}.png")
    # cv2.imshow('HelloWorld', img)
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(grey, 13, 15, 15) 



    edged = cv2.Canny(gray, 30, 200) 
    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None


    for c in contours:
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break


    if screenCnt is None:
        detected = 0
        print ("No contour detected")
        continue
        
    else:
        detected = 1

    if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)

    # cv2.imshow(f"car {i}",new_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected


In [28]:
img    = cv2.imread(f"archive/images/Cars1.png")
# cv2.imshow('HelloWorld', img)
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(grey, 13, 15, 15) 



edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None


for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break


if screenCnt is None:
    detected = 0
    print ("No contour detected")
    
else:
    detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)


(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("programming_fever's License Plate Recognition\n")
print("Detected license plate Number is:",text)
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

TesseractNotFoundError: c:\users\manu-\appdata\local\pip\cache\wheels\ed\8a\c6\40c7ec06c2dd3df636832537238128a1471bf9c1b6a3a9bf40\tesseract.exe is not installed or it's not in your PATH. See README file for more information.